In [203]:
import pandas as pd
import numpy as np
# from dateutil.parser import parser
# from dateutil.tz import gettz
from datetime import datetime
import time
import re
import json

In [2]:
!head -n 1 Data/tweets/part-00000

(1069728068015677440,MarkMillerITPro,Mon Dec 03 17:00:20 CST 2018,“Alexa open Night Light.” Depending on where you place an Echo this can be helpful for times when other sources of… https://t.co/pw9U675xwE,true,false,0,0,false,(,),1)


In [9]:
!wc -l Data/tweets/part-00000

94749 Data/tweets/part-00000


In [102]:
!wc -l Data/retweets/part-00000

174469 Data/retweets/part-00000


In [103]:
with open('Data/tweets/part-00000') as f:
    tweet_lines = f.readlines()
tweet_lines = [x.strip().split(',') for x in tweet_lines] 

In [104]:
with open('Data/retweets/part-00000') as f:
    retweet_lines = f.readlines()
retweet_lines = [x.strip().split(',') for x in retweet_lines] 

In [105]:
len(tweet_lines), len(retweet_lines)

(94749, 174469)

In [106]:
len(tweet_lines[0]), len(retweet_lines[0])

(12, 12)

In [162]:
start = time.time()
tweet_dict = {}
for tweet in tweet_lines:
    if tweet[0][1:] in tweet_dict:
        tweet_dict[tweet[0][1:]]['count'] += 1 
    else:
        tweet_dict[tweet[0][1:]] = {}
        tweet_dict[tweet[0][1:]]['count'] = 1
        tweet_dict[tweet[0][1:]]['retweet_count'] = int(tweet[6])
        tweet_dict[tweet[0][1:]]['fav_count'] = int(tweet[7])
    tweet_dict[tweet[0][1:]]['username'] = tweet[1]
    tweet_dict[tweet[0][1:]]['created'] = datetime.strptime(tweet[2][:19]+tweet[2][23:], '%a %b %d %H:%M:%S %Y')
    tweet_dict[tweet[0][1:]]['text'] = tweet[3]
    tweet_dict[tweet[0][1:]]['truncated'] = tweet[4]=='true'
    tweet_dict[tweet[0][1:]]['retweet'] = tweet[5]=='true'
    tweet_dict[tweet[0][1:]]['retweet_count'] = int(tweet[6])
    tweet_dict[tweet[0][1:]]['fav_count'] = int(tweet[7])
    tweet_dict[tweet[0][1:]]['sensitive'] = tweet[8]=='true'
    tweet_dict[tweet[0][1:]]['country'] = tweet[9][1:]
    tweet_dict[tweet[0][1:]]['place'] = tweet[10][:-1]
    tweet_dict[tweet[0][1:]]['hate_count'] = int(tweet[-1][:-1])
print(f"it took {time.time()-start} seconds to create dictionary")

it took 1.4272100925445557 seconds to create dictionary


- parser: it took 11.324670553207397 seconds to create dictionary
- datetime.strptime: it took 1.355278491973877 seconds to create dictionary

In [128]:
for i in retweet_lines:
    if len(i[2]) != 28:
        print(i)

In [133]:
retweet_lines[0][2][:19]+retweet_lines[0][2][23:]

'Mon Dec 03 15:38:39 2018'

In [129]:
retweet_lines[0][2].find("CST")

20

In [161]:
start = time.time()
retweet_dict = {}
for tweet in retweet_lines:
    if tweet[0][1:] in retweet_dict:
        retweet_dict[tweet[0][1:]]['count'] += 1 
        retweet_dict[tweet[0][1:]]['retweet_count'] += int(tweet[6])
        retweet_dict[tweet[0][1:]]['fav_count'] += int(tweet[7])
    else:
        retweet_dict[tweet[0][1:]] = {}
        retweet_dict[tweet[0][1:]]['count'] = 1
        retweet_dict[tweet[0][1:]]['retweet_count'] = int(tweet[6])
        retweet_dict[tweet[0][1:]]['fav_count'] = int(tweet[7])
    retweet_dict[tweet[0][1:]]['username'] = tweet[1]
    retweet_dict[tweet[0][1:]]['created'] = datetime.strptime(tweet[2][:19]+tweet[2][23:], '%a %b %d %H:%M:%S %Y')
    retweet_dict[tweet[0][1:]]['text'] = tweet[3]
    retweet_dict[tweet[0][1:]]['truncated'] = tweet[4]=='true'
    retweet_dict[tweet[0][1:]]['retweet'] = tweet[5]=='true'
    retweet_dict[tweet[0][1:]]['sensitive'] = tweet[8]=='true'
    retweet_dict[tweet[0][1:]]['country'] = tweet[9][1:]
    retweet_dict[tweet[0][1:]]['place'] = tweet[10][:-1]
    retweet_dict[tweet[0][1:]]['hate_count'] = int(tweet[-1][:-1])
print(f"it took {time.time()-start} seconds to create dictionary")

it took 2.6290180683135986 seconds to create dictionary


In [136]:
len(tweet_dict.keys()), len(retweet_dict.keys())

(94480, 82758)

In [137]:
# number of duplicated tweets
len(tweet_lines)-len(tweet_dict.keys())

269

In [138]:
# number of duplicated retweets
len(retweet_lines)-len(retweet_dict.keys())

91711

In [148]:
# get a few examples of duplicate retweets
for tweet in retweet_dict:
    if retweet_dict[tweet]['count'] > 500:
        print(tweet, retweet_dict[tweet]['count'])

1069370188934639616 575
1069784569975517184 515


In [152]:
# get a few examples of duplicate retweets
for tweet in retweet_dict:
    if retweet_dict[tweet]['hate_count'] > 80:
        print(tweet, retweet_dict[tweet]['hate_count'])

1069280664640311297 84
1069579640103100417 81
1069809990460891136 90
1069605669001723904 86
1069811482332983296 82
1069877363876528133 85
1068644947048443904 85


In [156]:
retweet_dict['1069280664640311297']

{'count': 1,
 'username': 'ACTJISUS',
 'created': datetime.datetime(2018, 12, 2, 11, 22, 31),
 'text': 'Bobby                          Bobbyas a                              with aBaby                             Baby https://t.co/NThoCJj6mX',
 'truncated': False,
 'retweet': False,
 'retweet_count': 1142,
 'fav_count': 1432,
 'sensitive': False,
 'country': '',
 'place': '',
 'hate_count': 84}

In [157]:
# check duplicated tweets
for tweet in tweet_lines:
    if tweet[0][1:] == '1069728994961055745':
        print(tweet)

['(1069728994961055745', 'ffemt01', 'Mon Dec 03 17:04:01 CST 2018', '@kimswallows What a lucky fan', 'false', 'false', '0', '0', 'false', '(', ')', '2)']
['(1069728994961055745', 'ffemt01', 'Mon Dec 03 17:04:01 CST 2018', '@kimswallows What a lucky fan', 'false', 'false', '0', '0', 'false', '(', ')', '2)']
['(1069728994961055745', 'ffemt01', 'Mon Dec 03 17:04:01 CST 2018', '@kimswallows What a lucky fan', 'false', 'false', '0', '0', 'false', '(', ')', '2)']
['(1069728994961055745', 'ffemt01', 'Mon Dec 03 17:04:01 CST 2018', '@kimswallows What a lucky fan', 'false', 'false', '0', '0', 'false', '(', ')', '2)']


In [ ]:
for tweet in retweet_lines:
    if tweet[0][1:] == '1069370188934639616':
        print(tweet)

In [206]:
with open('Data/tweets.json', 'w') as f:
    json.dump(tweet_dict, f, default=str)
with open('Data/retweets.json', 'w') as f:
    json.dump(retweet_dict, f, default=str)

In [232]:
tweet_df = pd.read_json('Data/tweets.json', orient='index', convert_dates=['created'])
retweet_df = pd.read_json('Data/retweets.json', orient='index', convert_dates=['created'])

In [212]:
tweet_df.shape, retweet_df.shape

((94480, 12), (82758, 12))

In [231]:
tweet_df.head()

,count,country,created,fav_count,hate_count,place,retweet,retweet_count,sensitive,text,truncated,username
2003-11-25 02:41:08.015677440,1,,2018-12-03 17:00:20,0,1,,0,0,0,“Alexa open Night Light.” Depending on where y...,1,MarkMillerITPro
2003-11-25 02:41:08.040777729,1,,2018-12-03 17:00:20,0,1,,0,0,0,this pic made me get up on time for the gym to...,0,IamPatriciaTone
2003-11-25 02:41:12.214016002,1,,2018-12-03 17:00:21,0,2,,0,0,0,@Dan23_92 And if a player has a bad game does...,1,MattHun67472111
2003-11-25 02:41:12.218411009,1,,2018-12-03 17:00:21,0,1,,0,0,0,Wonderful friends who love what DiscoveryBound...,1,DiscoveryBound
2003-11-25 02:41:16.421099521,1,,2018-12-03 17:00:22,0,1,,0,0,0,2018 was a good year for me. #unpopularopinion,0,kirstiwilder


In [214]:
retweet_df.head()

,count,country,created,fav_count,hate_count,place,retweet,retweet_count,sensitive,text,truncated,username
2003-11-24 20:58:30.423187457,110,,2018-12-03 15:38:39,5563932,1,,0,2013671,0,I’m so glad I met these boys and we chased dow...,1,Luke5SOS
2003-11-16 05:24:28.846317572,7,,2018-12-01 14:09:23,240278,1,,0,131707,0,if you didn't ask me don't assume. if u didn't...,1,xforcades2
2003-11-17 06:43:40.308058112,21,,2018-12-01 20:11:35,2594754,1,,0,778049,0,Raccoons really be the crackheads of the anima...,0,NoelSznn
2003-11-24 22:12:01.828544517,1,,2018-12-03 15:56:11,9,1,,0,3,0,"""Peachy~!""//AU!Peach where she got married to ...",1,_KoopaQueen_
2003-11-24 12:22:07.414099968,24,,2018-12-03 13:35:32,88806,2,,0,63855,0,"""Federal Employees Are Warned Not to Discuss T...",1,JamesOKeefeIII


In [246]:
by_hour = pd.DataFrame(tweet_df['hate_count'].groupby([tweet_df['created'].dt.date.rename('day'),tweet_df['created'].dt.hour.rename('hour')]).sum())

In [252]:
by_hour = pd.DataFrame(tweet_df['hate_count'].groupby(tweet_df['created'].dt.date).sum())

In [253]:
by_hour.reset_index(inplace=True)

In [255]:
by_hour

,created,hate_count
0,2018-12-03,83804
1,2018-12-04,66502


In [254]:
by_hour['hate_count']

0    83804
1    66502
Name: hate_count, dtype: int64

In [236]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv")

In [238]:
df['Date'][:10]

0    2015-02-17
1    2015-02-18
2    2015-02-19
3    2015-02-20
4    2015-02-23
5    2015-02-24
6    2015-02-25
7    2015-02-26
8    2015-02-27
9    2015-03-02
Name: Date, dtype: object

In [239]:
df['AAPL.Close'][:10]

0    127.830002
1    128.720001
2    128.449997
3    129.500000
4    133.000000
5    132.169998
6    128.789993
7    130.419998
8    128.460007
9    129.089996
Name: AAPL.Close, dtype: float64